In [1]:
import os

import flask
app = flask.Flask(__name__)
from flask import Flask, render_template, request

#load model preprocessing
import numpy as np
import pandas as pd
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.models
from keras.models import model_from_json

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load tokenizer for preprocessing
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load pre-trained model into memory
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into new model
loaded_model.load_weights("model.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
def prepData(text):
    # Convert to array
    textDataArray = [text]
    
    # Convert into list with word ids
    Features = tokenizer.texts_to_sequences(textDataArray)
    Features = pad_sequences(Features, 20, padding='post')
    
    return Features

In [4]:
loaded_model.compile(optimizer="Adam",loss='binary_crossentropy',metrics=['accuracy'])

In [5]:
textDataTest = 'Hey! How was dinner last night?'
textTokenizedTest = prepData(textDataTest)
out = np.asscalar(loaded_model.predict(textTokenizedTest))
out

0.022943690419197083

In [6]:
textDataTest = 'Your account has been hacked. Please call 1-800-0000 to speak with a representative'
textTokenizedTest = prepData(textDataTest)
out = np.asscalar(loaded_model.predict(textTokenizedTest))
out

0.9085490703582764

See above. The first message had a 2% probability being spam and the second had a 91% chance. 
Our pre-trained model is working!

In [7]:
# define a predict function as an endpoint 

@app.route('/', methods=['GET','POST'])
def predict():
    
    #whenever the predict method is called, we're going
    #to input the user entered text into the model
    #and return a prediction
    
    if request.method=='POST':
        
        #Note: for local run, we print information to debug and understand how app is working
        
        textData = request.form.get('text_entered')
        print(textData)

        Features = prepData(textData)
        print(Features)
        
        prediction = int((np.asscalar(loaded_model.predict(Features)))*100)
        print(prediction)
        
        #return prediction in new page
        return render_template('prediction.html', prediction=prediction)
    
    else:
        return render_template("search_page.html")   

Note: The block below may return error message "UnsupportedOperation: not writable". Follow instructions below to correct error if encountered.

Option 1 is to edit the echo and secho functions:

(1) find conda environment location in C:\Users\[user]\.conda\environments.txt (it is in file)

(2) go to this location on your computer

(3) go to anaconda3/site-packages/click folder on your computer

(4) open utils.py in this folder

(5) find “def echo” and change parameter “file=None” to “file=sys.stdout”

(6) open termui.py in this folder

(7) find “def secho” and change parameter “file=None” to “file=sys.stdout”


Option 2 is to refer to an older version of flask:

(1) open anaconda prompt

(2) “pip uninstall flask”

(3) “pip install flask=0.12.2”

In [ ]:
if __name__ == "__main__":
    
    #decide what port to run the app in
    port = int(os.environ.get('PORT', 5000))
       
    #run the app locally on the given port
    app.run(host='0.0.0.0', port=port) 

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
